In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import transforms, utils
from torchvision import transforms
import torch.optim as optim
from torch.autograd import Variable
from torch.utils import data as t_data


In [111]:
data_transforms = transforms.Compose([transforms.ToTensor()])
mnist_trainset = datasets.MNIST(root='./data', train=True,    
                           download=True, transform=data_transforms)
batch_size=4

dataloader_mnist_train = t_data.DataLoader(mnist_trainset, 
                                           batch_size=batch_size,
                                           shuffle=True
                                           )

In [129]:
dataloader_mnist_train

In [18]:
mnist_trainset[0]

(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 

In [22]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [112]:
def randomnoise():
    return torch.rand(batch_size,100)

In [131]:
class generator(nn.Module):
    def __init__(self,inp,out):
        super(generator,self).__init__()
        self.gen=nn.Sequential(
                               
                                
                               nn.Linear(inp,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,1000),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(1000,800),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(800,out)
                               
                                )
    def forward(self,x):
            x=self.gen(x)
            return x

In [132]:
class discriminator(nn.Module):
    def __init__(self,inp,out):
        super(discriminator,self).__init__()
        self.disc=nn.Sequential( 
                                
                               nn.Linear(inp,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,200),
                                 nn.ReLU(inplace=True),
                                nn.Linear(200,out),
                                nn.Sigmoid()
            )
    def forward(self,x):
            x=self.disc(x)
            return x

In [71]:
def plotit(a):
    a=a.detach()
    a=a.reshape(28,28)
    plt.imshow(a,cmap='binary')
    

    

In [72]:
generator

__main__.generator

In [133]:
gen=generator(100,784)

In [134]:
gen

generator(
  (gen): Sequential(
    (0): Conv2d(100, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Linear(in_features=40000, out_features=300, bias=True)
    (4): ReLU(inplace)
    (5): Linear(in_features=300, out_features=1000, bias=True)
    (6): ReLU(inplace)
    (7): Linear(in_features=1000, out_features=800, bias=True)
    (8): ReLU(inplace)
    (9): Linear(in_features=800, out_features=784, bias=True)
  )
)

In [135]:
discriminator


__main__.discriminator

In [136]:
dis=discriminator(784,1)

In [137]:
dis
d_steps = 100
g_steps = 100

criteriond1 = nn.BCELoss()
optimizerd1 = optim.SGD(dis.parameters(), lr=0.001, momentum=0.9)

criteriond2 = nn.BCELoss()
optimizerd2 = optim.SGD(gen.parameters(), lr=0.001, momentum=0.9)

printing_steps = 200

epochs = 50

In [ ]:
for epoch in range(epochs):
    
    if epoch%printing_steps==0:
        print(epoch)

    
    for d_step in range(d_steps):
        dis.zero_grad()
        
        
        for inp_real,_ in dataloader_mnist_train:
            inp_real_x = inp_real
            break

        inp_real_x = inp_real_x.reshape(4,784)
        dis_real_out = dis(inp_real_x)
        dis_real_loss = criteriond1(dis_real_out,Variable(torch.ones(batch_size,1)))
        dis_real_loss.backward()

       
        inp_fake_x_gen = randomnoise()
        dis_inp_fake_x = gen(inp_fake_x_gen).detach() 
        dis_fake_out = dis(dis_inp_fake_x)
        dis_fake_loss = criteriond1(dis_fake_out,Variable(torch.zeros(batch_size,1)))
        dis_fake_loss.backward()

        optimizerd1.step()
        
        
            
  
    for g_step in range(g_steps):
        gen.zero_grad()
        
        
        gen_inp = randomnoise()
        
        gen_out = gen(gen_inp)
        dis_out_gen_training = dis(gen_out)
        gen_loss = criteriond2(dis_out_gen_training,Variable(torch.ones(batch_size,1)))
        gen_loss.backward()
        
        optimizerd2.step()
        
    if epoch%printing_steps==0:
        plotit(gen_out[0])
        plotit(gen_out[1])
        plotit(gen_out[2])
        plotit(gen_out[3])
        print("\n\n")